In [26]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
#sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 
#sc.install_pypi_package("matplotlib", "https://pypi.org/simple") #Install matplotlib from given PyPI repository

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#load customers
customers  = sqlContext.read.json('s3://tfmbigdata/Files_Segunda_Entrega/Customers.json')

#load products
products  = sqlContext.read.json('s3://tfmbigdata/Files_Segunda_Entrega/Products.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
#load tickets

text = sc.textFile("s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part0.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part1.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part2.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part3.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part4.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part5.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part6.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part7.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part8.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part9.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part10.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part11.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part12.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part13.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part14.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part15.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part16.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part17.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part18.json,s3://tfmbigdata/Files_Segunda_Entrega/Tickets_part19.json")

tickets = spark.read.json(text)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
tickets.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

13238241